In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("Hitters.csv")
df = df.copy()
df.head()
#veri setimizde index problemi yok 


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [2]:
df.shape


(322, 20)

In [3]:
df.isnull().sum()
# salary değişkenlerimizde boş gözlemler bulunmaktadır 
# bunları ortalama ile dolduralım 


AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [4]:
df["Salary"].fillna(df.Salary.mean() , inplace = True)
# boş maaş değerlerini ortalama ile doldurduk aslında bunları ilerde tahmin ile dolduracağız

In [5]:
df.isnull().sum().any()
# datamızda boş değer var mı ? 


False

In [6]:
df.info()
# veri setimizdeki kategorik değişkenleri düzenliyelim 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
AtBat        322 non-null int64
Hits         322 non-null int64
HmRun        322 non-null int64
Runs         322 non-null int64
RBI          322 non-null int64
Walks        322 non-null int64
Years        322 non-null int64
CAtBat       322 non-null int64
CHits        322 non-null int64
CHmRun       322 non-null int64
CRuns        322 non-null int64
CRBI         322 non-null int64
CWalks       322 non-null int64
League       322 non-null object
Division     322 non-null object
PutOuts      322 non-null int64
Assists      322 non-null int64
Errors       322 non-null int64
Salary       322 non-null float64
NewLeague    322 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 50.4+ KB


# Kategorik değişkenleri modellemeye hazır hale getirme 

In [7]:
from sklearn.preprocessing import LabelEncoder 
cat_list = ["NewLeague" , "Division" , "League"] 

lbe = LabelEncoder()
for i in cat_list:
    df[i] = lbe.fit_transform(df[i])

In [8]:
df.head(3)
# tip dönüşümlerini başarılı bir şekilde uyguladık 

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,535.925882,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.000000,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.000000,0


# Bagimli ve bağımsız değişken ayrışımı 

In [9]:
X = df.drop("Salary"  , axis = 1) # bagımsız degisken 

In [10]:
y = df["Salary"] #bagimli değisken 

# Veri setini Train_Test olarak ayırma

In [11]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.20)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# doğru parcalamışız

(257, 19)
(257,)
(65, 19)
(65,)


# Veri setini standartlaştırma

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scaler = StandardScaler() 
# ölçeklendirme nesnesi anımlıyorum

In [15]:
X_train_scaled = scaler.fit_transform(X_train)
# X_train ile fit edip transfor ile tip dönüşümü yapıyorum

In [16]:
X_test_scaled = scaler.fit_transform(X_test)
# X_test ile fit edip transfor ile tip dönüşümü yapıyorum

# Model Kurma YSA


In [17]:
from sklearn.metrics import mean_squared_error 
from sklearn.neural_network import MLPRegressor

In [18]:
mlp = MLPRegressor()
mlp


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

- alpha = lambda 
- hidden_layer_size = katmanlardaki noron sayısı

In [19]:
mlp_model = mlp.fit(X_train_scaled , y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [20]:
y_pred = mlp_model.predict(X_test_scaled)

In [21]:
np.sqrt( mean_squared_error(y_test , y_pred) )


550.5731324929217

In [22]:

mlp_model.n_layers_
#katman sayısısı

3

In [23]:
mlp_model.hidden_layer_sizes
# gizili katmanlarının içindeki norun sayısı

(100,)

# Model tuning 

In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV



In [25]:
mlp = MLPRegressor()
mlp_params = {'alpha' : [0.1 , 0.01 ,0.03 ,0.005],
              'hidden_layer_sizes' : [(20,20), (100, 50 ,150)],
              'activation' : ['relu' ,'logistic'],
             }


In [26]:
import warnings
warnings.filterwarnings("ignore")

mlp_cv = GridSearchCV(mlp ,
                      mlp_params ,
                      cv = 3,n_jobs = -1 ,verbose = 2).fit(X_train_scaled , y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    5.8s finished


In [27]:
mlp_cv.best_params_ 
# en iyi paramatreleri aldık

{'activation': 'relu', 'alpha': 0.005, 'hidden_layer_sizes': (100, 50, 150)}

In [28]:
# Final Modeli

In [29]:
mlp_final = MLPRegressor(activation = "relu" , alpha= 0.005 , hidden_layer_sizes = (100 ,50 ,150)
                        ).fit(X_train_scaled , y_train)

In [30]:
y_pred = mlp_final.predict(X_test_scaled)

In [31]:
np.sqrt( mean_squared_error(y_test , y_pred) )

325.3336994763134